# Performance Metrics in Basketball: PER


---

_Co-Authored with Derek Topper_


So far we have studied offensive metrics for baseball.  One thing we have seen is that a lot of metrics are built as linear sums of positive and negative contributions.  We will dissect PER (Player Efficiency Rating) in this notebook and observe how it works as a metric for player performance.

PER is a comprehensive metric that includes defensive statistics as well as offensive statistics.  While we have so far tried to segregate the two parts of play in baseball, we'll ignore that for now.

Basketball has a lot of moving parts and so the challenge is to figure out what the positive and negative contributions a player can make are and how to value them.

This notebook focuses on calculating Player Efficiency Rating given Raw NBA Player Data. PER looks like a complex, nasty equation but this notebook will break it down and show how simple and powerful it can be.




<img src="http://www.mercurynews.com/wp-content/uploads/2017/02/sjm-stancalbkc-0218-01.jpg?w=620" style="width: 300px; height: 450px;" />


## Setup and Data

In [1]:
%run ../../utils/notebook_setup.py

Adding datascience helper tools to path...
Setting up Matplotlib...
Matplotlib imported as mpl
Matplotlib.pyplot imported as plt


In [2]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

### The Data

We'll be working the total stats of all players from the 2016-17 NBA season. The metrics contain each player's unique totals for statistics like Points, Assists, Rebounds, Blocks and Steals.

Here are the columns in the table below:
* Rk -- Rank
* Pos -- Position
* Age -- Age of Player at the start of February 1st of that season.
* Tm -- Team
* G -- Games
* GS -- Games Started
* MP -- Minutes Played
* FG -- Field Goals
* FGA -- Field Goal Attempts
* FG% -- Field Goal Percentage
* 3P -- 3-Point Field Goals
* 3PA -- 3-Point Field Goal Attempts
* 3P% -- 3-Point Field Goal Percentage
* 2P -- 2-Point Field Goals
* 2PA -- 2-point Field Goal Attempts
* 2P% -- 2-Point Field Goal Percentage
* eFG% -- Effective Field Goal Percentage
* FT -- Free Throws
* FTA -- Free Throw Attempts
* FT% -- Free Throw Percentage
* ORB -- Offensive Rebounds
* DRB -- Defensive Rebounds
* TRB -- Total Rebounds
* AST -- Assists
* STL -- Steals
* BLK -- Blocks
* TOV -- Turnovers
* PF -- Personal Fouls
* PTS -- Points


Our data has some players appearing more than once. That is because that player was traded or moved teams in the middle of the season. 

*For example:* Quincy Acy played 38 games total (TOT). Of those, 32 games were played for the Brooklyn Nets (BRK) and 6 were played for the Dallas Mavericks (DAL)

In [3]:
player_stats = Table().read_table('NBAPlayerStats2017.csv')
player_stats.show(5)

#### Remove TOT entries

In [4]:
player_stats = player_stats.where('Tm', are.not_equal_to('TOT'))

## PER

John Hollinger was an NBA columnist for ESPN.com for eight years and is currently the Vice President of Basketball Operations for the Memphis Grizzlies. While at ESPN, he coined many advanced metrics in order to quantify player and team performance, such as Player Efficiency Rating (PER), Offensive Efficiency, Defensive Efficiency and Pace Factor.

PER is a rating of a player’s per-minute statistical performance that Hollinger developed to make player comparisons easier, and has become a widely used standard over the past decade. Hollinger has described PER as the sum of <a href http://www.espn.com/nba/columns/story?columnist=hollinger_john&id=2850240> "all a player's positive accomplishments, subtracts the negative accomplishments, and returns a per-minute rating of a player's performance."</a>

As PER attempts to be an all-encompassing number that looks at positive accomplishments, such as field goals, free throws, 3-pointers, assists, rebounds, blocks and steals, and negative results, such as missed shots, turnovers and fouls. 

The formula adds positive stats and subtracts negative ones through a statistical point value system. The rating for each player is then adjusted to a per-minute basis so that no player is negatively impcted by lack of playing time. It is also adjusted for pace of play. In the end, PER serves as one number that attempts to create an overall player score.


### PER Formula
$$\ PER = \left ( uPER \times \frac{lgPace}{tmPace} \right ) \times \frac{15}{lguPER} $$

Where:
* ''uPER'' stands for unadjusted PER
* ''tm'', the prefix, indicating of team rather than of player;
* ''lg'', the prefix, indicating of league rather than of player;
* ''Pace'' is related to the style of play of a team. We'll also get there later.

The basic idea behind $uPER$ is the following:
\begin{align*}
uPER & = \frac{1}{min} \times \Bigg(\Bigg.\\
     & \quad\quad \text{Three Pointers Made} \\
     & \quad\quad + \text{Contributions from Assists} \\
     & \quad\quad + \text{Contributions from FGs} \\
     & \quad\quad + \text{Contributions from FTs} \\
     & \quad\quad - \text{Contributions from TOs} \\
     & \quad\quad - \text{Contributions from Missed FGs} \\
     & \quad\quad - \text{Contributions from Missed FTs} \\
     & \quad\quad + \text{Contributions from Def Rebounds} \\
     & \quad\quad + \text{Contributions from Off Rebounds} \\
     & \quad\quad + \text{Contributions from Steals} \\
     & \quad\quad + \text{Contributions from Blocks} \\
     & \quad\quad - \text{Contributions from Fouls} \\
     & \quad \Bigg.\Bigg)
\end{align*}

### The Components of $uPER$
Let's do the computation for a specific player: Steph Curry



In [5]:
curry = player_stats.where('Player', are.equal_to('Stephen Curry'))
curry

Rk   | Player        | Pos  | Age  | Tm   | G    | GS   | MP   | FG   | FGA  | FG%   | 3P   | 3PA  | 3P%   | 2P   | 2PA  | 2P%   | eFG% | FT   | FTA  | FT%   | ORB  | DRB  | TRB  | AST  | STL  | BLK  | TOV  | PF   | PTS
98   | Stephen Curry | PG   | 28   | GSW  | 79   | 79   | 2638 | 675  | 1443 | 0.468 | 324  | 789  | 0.411 | 351  | 654  | 0.537 | 0.58 | 325  | 362  | 0.898 | 61   | 292  | 353  | 524  | 142  | 17   | 239  | 183  | 1999

#### Three Pointers
Since three point shots are worth an extra point, we need to add in the extra point since it's not accounted for elsewhere.
$$
    \text{Three Pointers Made} = \mathit{3P}
$$

In [6]:
three_pt_contr = curry['3P'][0]
three_pt_contr

324

#### Assists

Assists were determined to have value of $2/3$: an assist directly leads to a bucket but you shouldn't get full credit for the bucket. You get two-thirds of a point from the field goal.

$$ \text{Contributions from Assists} = \frac23 \times AST$$

In [7]:
asts_contr = (2/3) * curry.column('AST')[0]
asts_contr

349.3333333333333

#### Field Goals

The FG might have been assisted.  If so, we need to discount the value of the FG to account for that.  We do that using a league average rate at which FGs actually have assists.  $\text{factor} \times \frac{tmAST}{tmFG}$ is meant to capture the expected number of FGs which were assisted with $\text{factor}$ driving that expected value.  This is a lot like some of the baseball calculations we've seen with stolen bases or GIDP.

$$
\text{Contributions from FGs} = \left ( 2 - \text{factor} \times \frac{tmAST}{tmFG} \right ) \times FG
$$
where 
$$
    \text{factor} = \frac{2}{3} - \left [ \left ( 0.5 \times \frac{lgAST}{lgFG} \right ) \div \left ( 2 \times \frac{lgFG}{lgFT} \right ) \right ]
$$

In [8]:
# Team values
team = player_stats.where('Tm', are.equal_to(curry.column('Tm')[0]))
tmAST = sum(team.column('AST'))
tmFG = (sum(team.column('FG')))

# League values
lgAST = sum(player_stats.column('AST'))
lgFG = sum(player_stats.column('FG'))
lgFT = sum(player_stats.column('FT'))

# Factor
factor = (2 / 3) - (0.5 * (lgAST / lgFG)) / (2 * (lgFG / lgFT))

# FGs
FG = curry.column('FG')[0]

fg_contr = (2 - factor * tmAST / tmFG) * FG
fg_contr

1064.1325273986868

#### Free Throws

As with FGs, we need to discount FTs by the expected number of times they were assisted.
$$
    \text{Contributions from FTs} = 0.5 \times FT \times \left ( 2 -  \frac{1}{3} \times \frac{tmAST}{tmFG} \right )
$$

In [9]:
FT = curry.column('FT')[0]
ft_contr = .5 * FT * (2 - tmAST / (3 * tmFG) )
ft_contr

286.79808418271045

#### Turnovers
Turnovers prevent a chance at scoring so we need to dock the value of a possession from the player's rating.
$$
    \text{Contributions from TOs} = VOP \times TO
$$
where $VOP$ is the value of a possession and is equal to
$$
    VOP = \frac{lgPTS}{lgFGA - lgORB + lgTO + 0.44 \times lgFTA}
$$
The denominator is an approximation to the number of possessions.

In [10]:
lgPTS = sum(player_stats.column('PTS'))
lgFGA = sum(player_stats.column('FGA'))
lgORB = sum(player_stats.column('ORB'))
lgTOV = sum(player_stats.column('TOV'))
lgFTA = sum(player_stats.column('FTA'))

vop = lgPTS / (lgFGA - lgORB + lgTOV + (.44 * lgFTA))
vop

1.0685413540268014

In [11]:
TO = curry['TOV'][0]
to_contr = TO * vop 
to_contr

255.38138361240553

#### MIssed FGs
We need to dock the player for missed FGs that got rebounded by the defense.  A missed shot and no offensive rebound means a loss in the value of a possession.
$$
    \text{Contributions from Missed FGs} = VOP \times DRBP \times \left(FGA - FG \right) 
$$
where 
$$
    DRBP = \frac{lgTRB - lgORB}{lgTRB}
$$

In [12]:
lgTRB = sum(player_stats.column('TRB'))
lgORB = sum(player_stats.column('ORB'))
drbp = (lgTRB - lgORB) / lgTRB
drbp

0.7670440745100238

In [13]:
FGA = curry.column('FGA')[0]
missedfg_contr = vop * drbp * (FGA - FG)
missedfg_contr

629.4668651329348

#### Missed FTs
We need to account for how missed FTs that didn't get rebounded by the offense led to a diminished value of the possession (not a full loss like a missed FG).  The arithmetic to account for this is given by,
$$
    \text{Contributions from Missed FTs} = VOP \times 0.44 \times \left(0.44 + 0.56 \times DRBP \right)
         \times \left(FTA - FT \right) 
$$

In [14]:
FTA = curry.column('FTA')[0]
missedft_contr = vop * .44 * (.44 + .56 * drbp) * (FTA - FT)
missedft_contr

15.126471672013666

#### Defensive Rebounds
Since you are gaining a possession for your team, you should be rewarded for your rebounds but at the rate at which the team offensive rebound.

$$
    \text{Contributions from Def Rebounds} = VOP \times \left(1 - DRBP \right) \times \left(TRB - ORB \right)
$$

In [15]:
TRB = curry.column('TRB')[0]
ORB = curry.column('ORB')[0]
drb_contr = vop * (1 - drbp) * (TRB - ORB)
drb_contr

72.68552769507475

#### Offensive Rebounds
Offensive rebounds lead to an extended possession so this should be rewarded but discounted by how often teams give up offensive rebounds.
$$
    \text{Contributions from Off Rebounds} = VOP \times DRBP \times ORB 
$$

In [16]:
orb_contr = vop * drbp * ORB
orb_contr

49.99671715248571

#### Steals
Steals lead to a possession for the team so reward with $VOP$
$$
\text{Contributions from Steals} = VOP \times STL
$$

In [17]:
STL = curry['STL'][0]
stl_contr = vop * STL
stl_contr

151.7328722718058

#### Blocks
Blocks are rewarded for gaining a possession at the rate at which they are rebounded.
$$
    \text{Contributions from Blocks} = VOP \times DRBP \times BLK 
$$

In [18]:
BLK = curry['BLK'][0]
blk_contr = vop * drbp * BLK
blk_contr

13.933511337577983

#### Fouls
Fouls lead to opposing points so you should be docked for giving up points.  You should only be docked for giving points above the expected value for those possessions.

\begin{align*}
    \text{Contributions from Fouls} & = \text{Total points from commmited fouls} - \text{Points expected on those possessions} \\
    & = 
    PF \times \frac{lgFT}{lgPF} - PF \times 0.44 \times \frac{lgFTA}{lgPF} \times VOP
\end{align*}

In [19]:
lgPF = sum(player_stats.column('PF'))

PF = curry['PF'][0]
foul_contr = PF * lgFT / lgPF - PF * .44 * lgFTA / lgPF * vop
foul_contr

64.12348330192515

#### Curry's $uPER$

We put together all the contributions and we get $uPER$.

In [20]:
MP = curry['MP'][0]
curry_uper = (
    three_pt_contr + asts_contr + fg_contr + ft_contr
    - to_contr - missedfg_contr - missedft_contr
    + drb_contr + orb_contr + stl_contr + blk_contr
    - foul_contr
) / MP
curry_uper

0.5111881613542061

### Computing PER

Recall the formula
$$\ PER = \left ( uPER \times \frac{lgPace}{tmPace} \right ) \times \frac{15}{lguPER} $$
where we now have $uPER$ for Steph Curry.

#### $lguPER$
We need to compute $uPER$ for every player.  We take the previous code and put it into a function to compute for every player.

In [21]:
def uPER(player_name, player_stats):
    player = player_stats.where('Player', are.equal_to(player_name))

    # Team values
    team = player_stats.where('Tm', are.equal_to(player['Tm'][0]))
    tmAST = sum(team.column('AST'))
    tmFG = (sum(team.column('FG')))

    # League values
    lgPTS = sum(player_stats.column('PTS'))
    lgFG = sum(player_stats.column('FG'))
    lgFGA = sum(player_stats.column('FGA'))
    lgAST = sum(player_stats.column('AST'))
    lgFT = sum(player_stats.column('FT'))
    lgFTA = sum(player_stats.column('FTA'))
    lgTRB = sum(player_stats.column('TRB'))
    lgORB = sum(player_stats.column('ORB'))
    lgTOV = sum(player_stats.column('TOV'))
    lgPF = sum(player_stats.column('PF'))

    # Values
    factor = (2 / 3) - (0.5 * (lgAST / lgFG)) / (2 * (lgFG / lgFT))
    vop = lgPTS / (lgFGA - lgORB + lgTOV + (.44 * lgFTA))
    drbp = (lgTRB - lgORB) / lgTRB

    # Stats
    MP = player['MP'][0]
    FG3 = player['3P'][0]
    FG = player.column('FG')[0]
    FGA = player.column('FGA')[0]
    AST = player.column('AST')[0]
    FT = player.column('FT')[0]
    FTA = player.column('FTA')[0]
    TRB = player.column('TRB')[0]
    ORB = player.column('ORB')[0]
    STL = player['STL'][0]
    TO = player['TOV'][0]
    BLK = player['BLK'][0]
    PF = player['PF'][0]

    # Contributions
    three_pt_contr = FG3
    asts_contr = (2/3) * AST
    fg_contr = (2 - factor * tmAST / tmFG) * FG
    ft_contr = .5 * FT * (2 - tmAST / (3 * tmFG))
    to_contr = TO * vop 
    missedfg_contr = vop * drbp * (FGA - FG)
    missedft_contr = vop * .44 * (.44 + .56 * drbp) * (FTA - FT)
    drb_contr = vop * (1 - drbp) * (TRB - ORB)
    orb_contr = vop * drbp * ORB
    stl_contr = vop * STL
    blk_contr = vop * drbp * BLK
    foul_contr = PF * lgFT / lgPF - PF * .44 * lgFTA / lgPF * vop

    player_uper = (
        three_pt_contr + asts_contr + fg_contr + ft_contr
        - to_contr - missedfg_contr - missedft_contr
        + drb_contr + orb_contr + stl_contr + blk_contr
        - foul_contr
    ) / MP
    return player_uper

In [22]:
# verify
uPER('Stephen Curry', player_stats) == curry_uper

True

#### Compute $uPER$ for each player

In [23]:
players = player_stats.column('Player')

array = make_array()
for player in players:
    array = np.append(array, uPER(player, player_stats))

player_stats = player_stats.with_column('uPER', array)
player_stats.show(20)

#### Team and League Pace
Team pace is stored in a separate file.

In [24]:
pace = Table().read_table('Pace.csv')
pace.sort('Pace', descending=True).show()

In [25]:
lgPace = pace.where('Team', 'League Average')['Pace'][0]
lgPace

96.4

In [26]:
player_stats = player_stats.join('Tm', pace, 'Team')
player_stats['Pace Factor'] = lgPace / player_stats['Pace']
player_stats.show(5)

In [27]:
PER = player_stats['uPER'] * player_stats['Pace Factor'] * 15 / np.mean(player_stats['uPER'])
player_stats['PER'] = PER

In [28]:
player_stats.sort('PER', descending=True).\
    where('MP', are.above_or_equal_to(1500)).\
    show(20)

#### Why we need to pace adjust

In [29]:
PER = player_stats['uPER'] * 15 / np.mean(player_stats['uPER'])
player_stats['PER_nopace'] = player_stats['uPER'] * 15 / np.mean(player_stats['uPER'])
player_stats['PER_diff'] = player_stats['PER_nopace'] - player_stats['PER']

In [30]:
player_stats.sort('PER_diff', descending=True).\
    where('MP', are.above_or_equal_to(1500)).\
    show(20)